### 加载模型

In [1]:
from transformers import AutoTokenizer, MusicgenForConditionalGeneration, AutoProcessor, EncodecModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# 这个模型是我们要训练的
model_name = "facebook/musicgen-medium"  # 可选：small, medium, large
# 初次使用记得去掉local_files_only=True
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
processor = AutoProcessor.from_pretrained(model_name, local_files_only=True)
model = MusicgenForConditionalGeneration.from_pretrained(model_name, local_files_only=True).half().to(device)
# model.half()解决精度问题报错

model.config.decoder.decoder_start_token_id = model.generation_config.decoder_start_token_id
print("decoder_start_token_id:", model.config.decoder.decoder_start_token_id)

# 这个模型用于将音频转化为 tokens
encodec_model = EncodecModel.from_pretrained("facebook/encodec_32khz", local_files_only=True).to(device)
encodec_model.eval()

model

d:\Anaconda\envs\NLP2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cuda
decoder_start_token_id: 2048


MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

### 加载数据集

In [2]:
import librosa
from datasets import Dataset
import os
import numpy as np

def process_data(batch):
    # 加载音频并标准化
    audio, sr = librosa.load(os.path.join("./data/datashare/", batch["location"]), sr=32000)
    
    # 将音频转化为 tokens
    with torch.no_grad():
        encoded_frames = encodec_model.encode(torch.tensor(audio, device=device).unsqueeze(0).unsqueeze(0))  # 输出码本 tokens

    # 取出 tokens（通常是 List[Tensor]，每个 Tensor 是 [batch, num_codebooks, frames]）
    audio_tokens = torch.stack([codebook.cpu() for codebook in encoded_frames.audio_codes]).squeeze(0).squeeze(0).transpose(0, 1)
    # [frames, num_codebooks]
    
    # 将文本转化为 tokens
    inputs = tokenizer(batch["main_caption"], return_tensors="pt")
    
    return {
        "input_ids": inputs["input_ids"].squeeze(dim=0), # shape: (text_length)
        "attention_mask": inputs["attention_mask"].squeeze(dim=0), # shape: (text_length)
        # "labels": [padded_audio] # shape: (channels, text_length). Here channels=1.
        "labels": audio_tokens
    }

def my_collator(batch):
    input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(i["input_ids"]) for i in batch], batch_first=True)
    mask = (input_ids != 0).bool()
    labels = torch.nn.utils.rnn.pad_sequence([torch.tensor(i["labels"]) for i in batch], batch_first=True)
    return {
        "input_ids": input_ids,
        "attention_mask": mask,
        "labels": labels
    }

In [3]:
from datasets import load_dataset
dataset = load_dataset("amaai-lab/MusicBench", split="test")

dataset = dataset.map(process_data, batched=False)
# dataset = dataset["train"].train_test_split(test_size=0.2)
dataset = dataset.train_test_split(0.2)

Parameter 'function'=<function process_data at 0x000001E8E03D3EC0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 800/800 [00:35<00:00, 22.68 examples/s]


### 设置lora参数

In [4]:
from peft import LoraConfig, get_peft_model

# 定义 LoRA 配置
lora_config = LoraConfig(
    r=8,                  # LoRA 的秩（Rank）
    lora_alpha=32,        # 缩放因子
    target_modules=["q_proj", "v_proj"],  # 目标模块（MusicGen 的注意力层）
    lora_dropout=0.05,    # Dropout 率
    bias="none",          # 不调整偏置
    task_type="CAUSAL_LM", # 因果语言模型任务
)

# 应用 LoRA
try:
    del model.peft_config
    lora_model.unload() # 用来防止之前运行得到的lora_model产生问题
    print("Old lora_model deleted.")
except:
    print("Initializing lora_model...")
    pass
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()  # 查看可训练参数（应远小于原始模型）
lora_model.save_pretrained("./outputs/musicgen-lora/initial_lora")

Initializing lora_model...
trainable params: 4,718,592 || all params: 2,009,969,730 || trainable%: 0.2348


### 设置训练参数

In [8]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./outputs/musicgen-lora",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    learning_rate=1e-4,  # LoRA 需要更高的学习率
    fp16=True,           # 混合精度训练
    logging_steps=100,
    save_steps=500,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=my_collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
dataset["train"][0]

{'dataset': 'MusicBench',
 'location': 'data/Z_0Ta-m_-hU.wav',
 'main_caption': 'This music is an intense instrumental.The tempo is fast with vigorous violin harmony that slows down to the accompaniment of a grim Piano harmony. The music is a Ritardando and has a grim, dark, intense,serious,bleak, dreary, and dangerous vibe to it. The chord sequence is Em7b5/D, D, Dm. The beat counts to 2. The tempo of this song is 169.0 beats per minute. The key is D minor.',
 'alt_caption': 'This music is an intense instrumental.The tempo is fast with vigorous violin harmony that slows down to the accompaniment of a grim Piano harmony. The music is a Ritardando and has a grim, dark, intense,serious,bleak, dreary, and dangerous vibe to it. The chord sequence is Em7b5/D, D, Dm. The beat counts to 2. The tempo of this song is 169.0 beats per minute. The key is D minor.',
 'prompt_aug': '',
 'prompt_ch': 'The chord sequence is Em7b5/D, D, Dm.',
 'prompt_bt': 'The beat counts to 2.',
 'prompt_bpm': 'The t

### 训练

In [10]:
trainer.train()

Step,Training Loss
100,7.934100
200,6.992800
300,6.829800
400,6.816800
500,6.838300
600,6.804200
700,6.849900
800,6.820900
900,6.731500
1000,6.788500


TrainOutput(global_step=1600, training_loss=6.873570556640625, metrics={'train_runtime': 492.692, 'train_samples_per_second': 6.495, 'train_steps_per_second': 3.247, 'total_flos': 3940732713401208.0, 'train_loss': 6.873570556640625, 'epoch': 5.0})